Try to add a relationship to temperature extremes

In [1]:
import utils
import json
import plotly.express as px


with open('sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)


In [2]:
average_household_constraints = {
    'freq': 'A',
    'unit': 'AVG',
    'geo': sub_codelists['geo_eu_countries']
}

nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': 'FC_OTH_HH_E',
    'siec': 'TOTAL',
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_eu_countries'],
    }

In [ ]:
average_household = utils.get_dataset_with_selection('ilc_lvph01', average_household_constraints)

demo_pjan = utils.get_dataset_with_selection('demo_pjan', demo_pjan_constraints)
demo_pjan.data.TIME_PERIOD  = demo_pjan.data.TIME_PERIOD  - 1

nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints)


In [ ]:
demo_pjan_df = demo_pjan.data.drop(columns=['OBS_FLAG'])
demo_pjan_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

average_household_df = average_household.data.drop(columns=['OBS_FLAG'])
average_household_df.set_index(['geo', 'TIME_PERIOD'], inplace=True)

number_of_households_df = demo_pjan_df.div(average_household_df)
display(number_of_households_df)


In [ ]:
nrg_bal_df = nrg_bal.data.set_index(['geo', 'TIME_PERIOD'])

display(nrg_bal.data)

In [6]:
consumption_per_household = nrg_bal_df.div(number_of_households_df)

In [ ]:
consumption_per_household.reset_index(inplace=True)
consumption_per_household['OBS_VALUE'] = consumption_per_household['OBS_VALUE'] * 1000
consumption_per_household.rename(columns={'OBS_VALUE': 'consumption_per_household_MWh'}, inplace=True)

fig = px.line(consumption_per_household, x='TIME_PERIOD', y='consumption_per_household_MWh', color='geo')
fig.show()
